Endemic Animal WebScraper

In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, ElementNotInteractableException

# Path to your chromedriver executable
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests

In [28]:
# Path to your chromedriver executable
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)

animal_name = "philippine flying lemur"
animal_name_search = animal_name.replace(" ", "+")

animal_link = f"https://www.google.com/search?q={animal_name_search}&source=lnms&tbm=isch"

driver.get(animal_link)
time.sleep(10)

# Continuously scroll down until the bottom of the page is reached
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

a = input("Waiting for user to start...")
time.sleep(10)
driver.execute_script("window.scrollTo(0,0);")

elements = driver.find_elements(By.XPATH, "//div[@class='eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd']")
len_elements = len(elements)

print(f"Found {len_elements} image containers")

for i in range(len_elements):
    if i >= len_elements:
        print("Finished")
        break

    try:
        element = elements[i]
        
        # Scroll element into view
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        
        # Locate the inner element that contains the text
        elem = driver.find_element(By.XPATH, f"(//div[@class='eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd']//div[@class='toI8Rb OSrXXb'])[{i+1}]")
        
        # Check if the text in 'elem' contains the value of 'animal_name'
        if animal_name.lower() in elem.text.lower():  # Using lower() to make the comparison case-insensitive
            print(f"Found matching element with text: {elem.text}")
            
            # Wait until the element is clickable
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f"(//div[@class='eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd'])[{i+1}]"))
            )

            # Click on the element
            element.click()
            print(f"Clicked on element {i+1} with matching text: {elem.text}")
        else:
            print(f"No match found in element {i+1}: {elem.text}")
        
    except (ElementNotInteractableException, ElementClickInterceptedException) as e:
        print(f"Could not click element {i+1}: {str(e)}")
        continue

    time.sleep(2)  # Wait a bit before moving to the next element


time.sleep(10)
driver.quit()

Found 300 image containers
Found matching element with text: Philippine flying lemur - Wikipedia
Clicked on element 1 with matching text: Philippine flying lemur - Wikipedia
Found matching element with text: Philippine flying lemur | mammal ...
Clicked on element 2 with matching text: Philippine flying lemur | mammal ...
Found matching element with text: Philippine flying lemur (Cynocephalus ...
Clicked on element 3 with matching text: Philippine flying lemur (Cynocephalus ...
Found matching element with text: Philippine flying lemur (Cynocephalus ...
Clicked on element 4 with matching text: Philippine flying lemur (Cynocephalus ...
No match found in element 5: Philippine colugo (Cynocephalus… | Flickr
Found matching element with text: Philippine flying lemur (Cynocephalus ...
Clicked on element 6 with matching text: Philippine flying lemur (Cynocephalus ...
Found matching element with text: PHILIPPINE FLYING LEMUR Cynocephalus ...
Clicked on element 7 with matching text: PHILIPPINE FL